In [18]:
from machine_learning.xgb_def import XGBOptimizer
import pandas as pd
from opt_funct.data_fuct import functions
from sklearn.model_selection import train_test_split

In [16]:
df = functions.clean_ds(pd.read_csv('./data/aapl_1h_train.csv'))
df

,Timestamp,Gmtoffset,Datetime,Open,High,Low,Close,Volume,X_t-1,X_t-2,X_t-3,Pt_5,RSI,Y_BUY,Y_SELL
0,1602509400,0,2020-10-12 13:30:00,120.059997,121.330101,119.284500,120.919998,48082766.0,NaN,NaN,NaN,124.314102,NaN,1,0
1,1602513000,0,2020-10-12 14:30:00,120.919998,122.639999,120.769996,122.529998,27028067.0,120.919998,NaN,NaN,124.419998,NaN,0,0
2,1602516600,0,2020-10-12 15:30:00,122.525001,123.629997,122.290000,123.261001,35722193.0,122.529998,120.919998,NaN,123.757202,NaN,0,0
3,1602520200,0,2020-10-12 16:30:00,123.260002,124.000000,123.040000,123.930000,22891159.0,123.261001,122.529998,120.919998,124.430000,NaN,0,0
4,1602523800,0,2020-10-12 17:30:00,124.089996,125.180000,123.910003,125.050003,31443470.0,123.930000,123.261001,122.529998,123.080001,NaN,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
390,1609432200,0,2020-12-31 16:30:00,132.529998,132.800003,131.720001,132.619995,13538948.0,132.529998,132.884994,133.679992,NaN,50.983206,0,0
391,1609435800,0,2020-12-31 17:30:00,132.619995,132.717605,132.310104,132.560806,7474176.0,132.619995,132.529998,132.884994,NaN,50.783652,0,0
392,1609439400,0,2020-12-31 18:30:00,132.565002,132.889999,132.009994,132.764999,9408857.0,132.560806,132.619995,132.529998,NaN,51.463321,0,0
393,1609443000,0,2020-12-31 19:30:00,132.770004,133.179992,132.565994,132.800003,10147999.0,132.764999,132.560806,132.619995,NaN,51.582189,0,0


In [19]:
close_data = df[['Close', 'X_t-1', 'X_t-2', 'X_t-3', 'RSI', 'Y_BUY']]
x = close_data.drop(columns=['Y_BUY'])
y = close_data[['Y_BUY']]
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42)
XGBOptimizer(x_train, y_train, x_test, y_test).xgb_optuna()

[I 2024-03-06 19:45:12,509] A new study created in memory with name: no-name-83ac3e10-c3f2-4eca-a738-9e15ec9cdd96
/Users/rixon/Desktop/ITESO/11/Trading/Proyecto3/003_ML/machine_learning/xgb_def.py:19: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  'learning_rate': trial.suggest_loguniform('learning_rate', 0.01, 0.3),
/Users/rixon/Desktop/ITESO/11/Trading/Proyecto3/003_ML/machine_learning/xgb_def.py:21: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  'gamma': trial.suggest_loguniform('gamma', 0.1, 1),
/Users/rixon/Desktop/ITESO/11/Trading/Proyecto3/003_ML/machine_learning/xgb_def.py:22: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be re

XGBoostError: [19:45:12] /Users/runner/work/xgboost/xgboost/src/metric/metric.cc:49: Unknown metric function binary:logistic
Stack trace:
  [bt] (0) 1   libxgboost.dylib                    0x00000001697f0994 dmlc::LogMessageFatal::~LogMessageFatal() + 124
  [bt] (1) 2   libxgboost.dylib                    0x00000001699705c4 xgboost::Metric::Create(std::__1::basic_string<char, std::__1::char_traits<char>, std::__1::allocator<char>> const&, xgboost::Context const*) + 124
  [bt] (2) 3   libxgboost.dylib                    0x00000001699461d8 xgboost::LearnerConfiguration::ConfigureMetrics(std::__1::vector<std::__1::pair<std::__1::basic_string<char, std::__1::char_traits<char>, std::__1::allocator<char>>, std::__1::basic_string<char, std::__1::char_traits<char>, std::__1::allocator<char>>>, std::__1::allocator<std::__1::pair<std::__1::basic_string<char, std::__1::char_traits<char>, std::__1::allocator<char>>, std::__1::basic_string<char, std::__1::char_traits<char>, std::__1::allocator<char